<a href="https://colab.research.google.com/github/SaumyaGupta-99/text-generation/blob/master/textgeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [0]:
filename='/content/gdrive/My Drive/textgeneration/wonderland.txt'
raw_text=open(filename,'r',encoding='utf-8',errors='ignore').read()
raw_text=raw_text.lower()

In [0]:
chars=sorted(list(set(raw_text)))
char_to_int=dict((c,i) for i,c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print(int_to_char)

{'\n': 0, ' ': 1, '!': 2, '(': 3, ')': 4, '*': 5, ',': 6, '-': 7, '.': 8, ':': 9, ';': 10, '?': 11, '[': 12, ']': 13, '_': 14, 'a': 15, 'b': 16, 'c': 17, 'd': 18, 'e': 19, 'f': 20, 'g': 21, 'h': 22, 'i': 23, 'j': 24, 'k': 25, 'l': 26, 'm': 27, 'n': 28, 'o': 29, 'p': 30, 'q': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'w': 37, 'x': 38, 'y': 39, 'z': 40}
{0: '\n', 1: ' ', 2: '!', 3: '(', 4: ')', 5: '*', 6: ',', 7: '-', 8: '.', 9: ':', 10: ';', 11: '?', 12: '[', 13: ']', 14: '_', 15: 'a', 16: 'b', 17: 'c', 18: 'd', 19: 'e', 20: 'f', 21: 'g', 22: 'h', 23: 'i', 24: 'j', 25: 'k', 26: 'l', 27: 'm', 28: 'n', 29: 'o', 30: 'p', 31: 'q', 32: 'r', 33: 's', 34: 't', 35: 'u', 36: 'v', 37: 'w', 38: 'x', 39: 'y', 40: 'z'}


In [0]:
n_chars=len(raw_text)
n_vocab=len(chars)
print(n_chars)
print(n_vocab)

141418
41


In [0]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_pattern = len(dataX)
print ("Total Patterns: ", n_pattern)

Total Patterns:  141318


In [0]:
print(dataX[:10])
print(dataY[:10])

[[15, 26, 23, 17, 19, 33, 1, 15, 18, 36, 19, 28, 34, 35, 32, 19, 33, 1, 23, 28, 1, 37, 29, 28, 18, 19, 32, 26, 15, 28, 18, 0, 0, 26, 19, 37, 23, 33, 1, 17, 15, 32, 32, 29, 26, 26, 0, 0, 17, 22, 15, 30, 34, 19, 32, 1, 23, 8, 1, 18, 29, 37, 28, 1, 34, 22, 19, 1, 32, 15, 16, 16, 23, 34, 7, 22, 29, 26, 19, 0, 0, 15, 26, 23, 17, 19, 1, 37, 15, 33, 1, 16, 19, 21, 23, 28, 28, 23, 28, 21], [26, 23, 17, 19, 33, 1, 15, 18, 36, 19, 28, 34, 35, 32, 19, 33, 1, 23, 28, 1, 37, 29, 28, 18, 19, 32, 26, 15, 28, 18, 0, 0, 26, 19, 37, 23, 33, 1, 17, 15, 32, 32, 29, 26, 26, 0, 0, 17, 22, 15, 30, 34, 19, 32, 1, 23, 8, 1, 18, 29, 37, 28, 1, 34, 22, 19, 1, 32, 15, 16, 16, 23, 34, 7, 22, 29, 26, 19, 0, 0, 15, 26, 23, 17, 19, 1, 37, 15, 33, 1, 16, 19, 21, 23, 28, 28, 23, 28, 21, 1], [23, 17, 19, 33, 1, 15, 18, 36, 19, 28, 34, 35, 32, 19, 33, 1, 23, 28, 1, 37, 29, 28, 18, 19, 32, 26, 15, 28, 18, 0, 0, 26, 19, 37, 23, 33, 1, 17, 15, 32, 32, 29, 26, 26, 0, 0, 17, 22, 15, 30, 34, 19, 32, 1, 23, 8, 1, 18, 29, 37, 28

In [0]:
X = numpy.reshape(dataX, (n_pattern, seq_length, 1))
X = X / float(n_vocab)
y=np_utils.to_categorical(dataY)
print(X)
print(y)

[[[0.36585366]
  [0.63414634]
  [0.56097561]
  ...
  [0.56097561]
  [0.68292683]
  [0.51219512]]

 [[0.63414634]
  [0.56097561]
  [0.41463415]
  ...
  [0.68292683]
  [0.51219512]
  [0.02439024]]

 [[0.56097561]
  [0.41463415]
  [0.46341463]
  ...
  [0.51219512]
  [0.02439024]
  [0.82926829]]

 ...

 [[0.56097561]
  [0.7804878 ]
  [0.02439024]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.7804878 ]
  [0.02439024]
  [0.80487805]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.02439024]
  [0.80487805]
  [0.56097561]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]]
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
filepath="/content/gdrive/My Drive/textgeneration/weights-improvement-47-1.2219-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/50





141318/141318 [==============================] - 839s 6ms/step - loss: 2.7414

Epoch 00001: loss improved from inf to 2.74141, saving model to /content/gdrive/My Drive/textgeneration/weights-improvement-47-1.2219-bigger.hdf5
Epoch 2/50
141318/141318 [==============================] - 840s 6ms/step - loss: 2.3837

Epoch 00002: loss improved from 2.74141 to 2.38371, saving model to /content/gdrive/My Drive/textgeneration/weights-improvement-47-1.2219-bigger.hdf5
Epoch 3/50
141318/141318 [==============================] - 845s 6ms/step - loss: 2.1760

Epoch 00003: loss improved from 2.38371 to 2.17602, saving model to /content/gdrive/My Drive/textgeneration/weights-improvement-47-1.2219-bigger.hdf5
Epoch 4/50
141318/141318 [==============================] - 83

KeyboardInterrupt: ignored

In [0]:
start = numpy.random.randint(0, len(dataX)-1)
pattern=dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" g the time! off with his
head!

how dreadfully savage! exclaimed alice.

and ever since that, the ha "


In [0]:
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

tter was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the thing was the